In [6]:
import os
import re
import praw
import time
import string
import pyLDAvis
import numpy as np
import pandas as pd
import seaborn as sns
import pyLDAvis.lda_model
from tabulate import tabulate
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
reddit = praw.Reddit(
    client_id="OJtDOFbkvxW9oodT0qSozw",
    client_secret="6unjzbEWPaQoomY7uRYq0mQIia80IQ",
    user_agent="moon_rover69"
)

combined_data = []

# subreddit_names = ["UkraineRussiaReport", "RussiaUkraineWar", "RussiaUkraineWar2022", "UkrainianConflict"]
subreddit_names = ["UkraineRussiaReport"]
for subreddit_name in subreddit_names:
    subreddit = reddit.subreddit(subreddit_name)

    print("Fetching upto 1000 posts and top 50 comments each...")

    for idx, submission in enumerate(subreddit.new(limit=1000)):
        # Add the post itself
        print(f"Fetching comments for post: {idx}: {submission.title}")
        combined_data.append({
            'id': submission.id,
            'type': 'submission',
            'title': submission.title,
            'selftext': submission.selftext,
            'created_utc': submission.created_utc
        })

    # Fetch comments
        submission.comments.replace_more()
        for comment in submission.comments.list()[:50]:
            combined_data.append({
                'id': comment.id,
                'type': 'comment',
                'title': None,
                'selftext': comment.body,
                'created_utc': comment.created_utc
            })

        if (idx + 1) % 100 == 0:
            print(f"{idx + 1} posts processed...")
            time.sleep(1)

df = pd.DataFrame(combined_data)
file_path = 'reddit_combined_posts_and_comments.csv'
# If the file doesn't exist yet, write header; otherwise, append without header
write_header = not os.path.exists(file_path)
df.to_csv(
    "reddit_combined_posts_and_comments.csv",
    mode='a',
    header=write_header,
    index=False
)

print("✅ Done! Data saved with matching format.")

Fetching upto 1000 posts and top 50 comments each...
Fetching comments for post: 0: UA POV: NATO boss Mark Rutte says Russia is producing ammo 4x than all of NATO. He warns that by 2027–2032, Russia might do something silly when they gain enough strength. He also wants to let Putin know that any attack now or later would trigger a devastating NATO reaction
Fetching comments for post: 1: UA POV: US President Trump believes that if Russia had remained in the G8 (now G7), this war probably wouldn’t have happened. He said it would've been alot better if Russia was in.
Fetching comments for post: 2: RU POV: The moment Russian air defenses shot down a Ukrainian drone over the Moscow region
Fetching comments for post: 3: UA POV: A unit of the Russian GRU has gained access to 10,000 CCTV cameras, located near military facilities and railway stations to track the movement of goods to Ukraine. 80% of hacked cameras are located in Ukraine, 10% in Romania, 4% in Poland, 2.8% in Hungary, and 1.7% i